In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pefile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow

In [ ]:
import os
import pandas as pd
import numpy
import pickle
import pefile
import sklearn.ensemble as ek
from sklearn.model_selection import train_test_split
from sklearn import tree, linear_model
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/SEM6_Project/Malicious_File/PE_Malicious_File_Detection/malware_data.csv',sep='|')
dataset.groupby(dataset['legitimate']).size()

legitimate
0    96724
1    41323
dtype: int64

In [ ]:
dataset

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272727,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138042,VirusShare_8e292b418568d6e7b87f2a32aee7074b,8e292b418568d6e7b87f2a32aee7074b,332,224,258,11,0,205824,223744,0,...,7,4.122736,1.370260,7.677091,14900.714286,16,81654,72,0,0
138043,VirusShare_260d9e2258aed4c8a3bbd703ec895822,260d9e2258aed4c8a3bbd703ec895822,332,224,33167,2,25,37888,185344,0,...,26,3.377663,2.031619,5.050074,6905.846154,44,67624,0,15,0
138044,VirusShare_8d088a51b7d225c9f5d11d239791ec3f,8d088a51b7d225c9f5d11d239791ec3f,332,224,258,10,0,118272,380416,0,...,22,6.825406,2.617026,7.990487,14981.909091,48,22648,72,14,0
138045,VirusShare_4286dccf67ca220fe67635388229a9f3,4286dccf67ca220fe67635388229a9f3,332,224,33166,2,25,49152,16896,0,...,10,3.421627,2.060964,4.739744,601.600000,16,2216,0,0,0


# ANN Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Split features and target variable
X = dataset.drop(['Name', 'md5', 'legitimate'], axis=1)
y = dataset['legitimate']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate the model on the test dataset
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
y_pred = np.squeeze(y_pred)  # Convert predictions to 1D array


# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


Epoch 1/20
1381/1381 [==============================] - 5s 3ms/step - loss: 0.0581 - accuracy: 0.9841 - val_loss: 0.0424 - val_accuracy: 0.9866
Epoch 2/20
1381/1381 [==============================] - 4s 3ms/step - loss: 0.0379 - accuracy: 0.9884 - val_loss: 0.0689 - val_accuracy: 0.9835
Epoch 3/20
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0312 - accuracy: 0.9899 - val_loss: 0.0390 - val_accuracy: 0.9882
Epoch 4/20
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0284 - accuracy: 0.9904 - val_loss: 0.0366 - val_accuracy: 0.9889
Epoch 5/20
1381/1381 [==============================] - 4s 3ms/step - loss: 0.0270 - accuracy: 0.9907 - val_loss: 0.0354 - val_accuracy: 0.9887
Epoch 6/20
1381/1381 [==============================] - 4s 3ms/step - loss: 0.0255 - accuracy: 0.9911 - val_loss: 0.0333 - val_accuracy: 0.9889
Epoch 7/20
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0240 - accuracy: 0.9917 - val_loss: 0.0361 - val_accuracy:

In [ ]:
# Save the model to a file
model.save('ANN_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
%%writefile Final_Testing.py
"""
This file extracts the required information of a given file using the PE library.
"""

import pefile
import os
import array
import math
import pickle
import sys
import numpy as np
from tensorflow.keras.models import load_model

def get_entropy(data):
    if len(data) == 0:
        return 0.0
    occurrences = array.array('L', [0] * 256)
    for x in data:
        occurrences[x if isinstance(x, int) else ord(x)] += 1
    entropy = 0
    for x in occurrences:
        if x:
            p_x = float(x) / len(data)
            entropy -= p_x * math.log(p_x, 2)
    return entropy


def get_resources(pe):
    """Extract resources: [entropy, size]"""
    resources = []
    if hasattr(pe, 'DIRECTORY_ENTRY_RESOURCE'):
        try:
            for resource_type in pe.DIRECTORY_ENTRY_RESOURCE.entries:
                if hasattr(resource_type, 'directory'):
                    for resource_id in resource_type.directory.entries:
                        if hasattr(resource_id, 'directory'):
                            for resource_lang in resource_id.directory.entries:
                                data = pe.get_data(resource_lang.data.struct.OffsetToData,
                                                   resource_lang.data.struct.Size)
                                size = resource_lang.data.struct.Size
                                entropy = get_entropy(data)

                                resources.append([entropy, size])
        except Exception as e:
            return resources
    return resources

def get_version_info(pe):
    """Return version infos"""
    res = {}
    for fileinfo in pe.FileInfo:
        if fileinfo.Key == 'StringFileInfo':
            for st in fileinfo.StringTable:
                for entry in st.entries.items():
                    res[entry[0]] = entry[1]
        if fileinfo.Key == 'VarFileInfo':
            for var in fileinfo.Var:
                res[var.entry.items()[0][0]] = var.entry.items()[0][1]
    if hasattr(pe, 'VS_FIXEDFILEINFO'):
        res['flags'] = pe.VS_FIXEDFILEINFO.FileFlags
        res['os'] = pe.VS_FIXEDFILEINFO.FileOS
        res['type'] = pe.VS_FIXEDFILEINFO.FileType
        res['file_version'] = pe.VS_FIXEDFILEINFO.FileVersionLS
        res['product_version'] = pe.VS_FIXEDFILEINFO.ProductVersionLS
        res['signature'] = pe.VS_FIXEDFILEINFO.Signature
        res['struct_version'] = pe.VS_FIXEDFILEINFO.StrucVersion
    return res

def extract_infos(fpath):
    res = {}
    pe = pefile.PE(fpath)
    res['Machine'] = pe.FILE_HEADER.Machine
    res['SizeOfOptionalHeader'] = pe.FILE_HEADER.SizeOfOptionalHeader
    res['Characteristics'] = pe.FILE_HEADER.Characteristics
    res['MajorLinkerVersion'] = pe.OPTIONAL_HEADER.MajorLinkerVersion
    res['MinorLinkerVersion'] = pe.OPTIONAL_HEADER.MinorLinkerVersion
    res['SizeOfCode'] = pe.OPTIONAL_HEADER.SizeOfCode
    res['SizeOfInitializedData'] = pe.OPTIONAL_HEADER.SizeOfInitializedData
    res['SizeOfUninitializedData'] = pe.OPTIONAL_HEADER.SizeOfUninitializedData
    res['AddressOfEntryPoint'] = pe.OPTIONAL_HEADER.AddressOfEntryPoint
    res['BaseOfCode'] = pe.OPTIONAL_HEADER.BaseOfCode
    try:
        res['BaseOfData'] = pe.OPTIONAL_HEADER.BaseOfData
    except AttributeError:
        res['BaseOfData'] = 0
    res['ImageBase'] = pe.OPTIONAL_HEADER.ImageBase
    res['SectionAlignment'] = pe.OPTIONAL_HEADER.SectionAlignment
    res['FileAlignment'] = pe.OPTIONAL_HEADER.FileAlignment
    res['MajorOperatingSystemVersion'] = pe.OPTIONAL_HEADER.MajorOperatingSystemVersion
    res['MinorOperatingSystemVersion'] = pe.OPTIONAL_HEADER.MinorOperatingSystemVersion
    res['MajorImageVersion'] = pe.OPTIONAL_HEADER.MajorImageVersion
    res['MinorImageVersion'] = pe.OPTIONAL_HEADER.MinorImageVersion
    res['MajorSubsystemVersion'] = pe.OPTIONAL_HEADER.MajorSubsystemVersion
    res['MinorSubsystemVersion'] = pe.OPTIONAL_HEADER.MinorSubsystemVersion
    res['SizeOfImage'] = pe.OPTIONAL_HEADER.SizeOfImage
    res['SizeOfHeaders'] = pe.OPTIONAL_HEADER.SizeOfHeaders
    res['CheckSum'] = pe.OPTIONAL_HEADER.CheckSum
    res['Subsystem'] = pe.OPTIONAL_HEADER.Subsystem
    res['DllCharacteristics'] = pe.OPTIONAL_HEADER.DllCharacteristics
    res['SizeOfStackReserve'] = pe.OPTIONAL_HEADER.SizeOfStackReserve
    res['SizeOfStackCommit'] = pe.OPTIONAL_HEADER.SizeOfStackCommit
    res['SizeOfHeapReserve'] = pe.OPTIONAL_HEADER.SizeOfHeapReserve
    res['SizeOfHeapCommit'] = pe.OPTIONAL_HEADER.SizeOfHeapCommit
    res['LoaderFlags'] = pe.OPTIONAL_HEADER.LoaderFlags
    res['NumberOfRvaAndSizes'] = pe.OPTIONAL_HEADER.NumberOfRvaAndSizes

    # Sections
    res['SectionsNb'] = len(pe.sections)
    entropy = list(map(lambda x: x.get_entropy(), pe.sections))
    res['SectionsMeanEntropy'] = sum(entropy)/float(len(entropy))
    res['SectionsMinEntropy'] = min(entropy)
    res['SectionsMaxEntropy'] = max(entropy)
    raw_sizes = list(map(lambda x: x.SizeOfRawData, pe.sections))
    res['SectionsMeanRawsize'] = sum(raw_sizes)/float(len(raw_sizes))
    res['SectionsMinRawsize'] = min(raw_sizes)
    res['SectionsMaxRawsize'] = max(raw_sizes)
    virtual_sizes = list(map(lambda x: x.Misc_VirtualSize, pe.sections))
    res['SectionsMeanVirtualsize'] = sum(virtual_sizes)/float(len(virtual_sizes))
    res['SectionsMinVirtualsize'] = min(virtual_sizes)
    res['SectionMaxVirtualsize'] = max(virtual_sizes)

    # Imports
    try:
        res['ImportsNbDLL'] = len(pe.DIRECTORY_ENTRY_IMPORT)
        imports = sum([x.imports for x in pe.DIRECTORY_ENTRY_IMPORT], [])
        res['ImportsNb'] = len(imports)
        res['ImportsNbOrdinal'] = len(list(filter(lambda x: x.name is None, imports)))
    except AttributeError:
        res['ImportsNbDLL'] = 0
        res['ImportsNb'] = 0
        res['ImportsNbOrdinal'] = 0

    # Exports
    try:
        res['ExportNb'] = len(pe.DIRECTORY_ENTRY_EXPORT.symbols)
    except AttributeError:
        # No export
        res['ExportNb'] = 0

    # Resources
    resources = get_resources(pe)
    res['ResourcesNb'] = len(resources)
    if len(resources) > 0:
        entropy = list(map(lambda x: x[0], resources))
        res['ResourcesMeanEntropy'] = sum(entropy)/float(len(entropy))
        res['ResourcesMinEntropy'] = min(entropy)
        res['ResourcesMaxEntropy'] = max(entropy)
        sizes = list(map(lambda x: x[1], resources))
        res['ResourcesMeanSize'] = sum(sizes)/float(len(sizes))
        res['ResourcesMinSize'] = min(sizes)
        res['ResourcesMaxSize'] = max(sizes)
    else:
        res['ResourcesNb'] = 0
        res['ResourcesMeanEntropy'] = 0
        res['ResourcesMinEntropy'] = 0
        res['ResourcesMaxEntropy'] = 0
        res['ResourcesMeanSize'] = 0
        res['ResourcesMinSize'] = 0
        res['ResourcesMaxSize'] = 0

    # Load configuration size
    try:
        res['LoadConfigurationSize'] = pe.DIRECTORY_ENTRY_LOAD_CONFIG.struct.Size
    except AttributeError:
        res['LoadConfigurationSize'] = 0

    # Version configuration size
    try:
        version_infos = get_version_info(pe)
        res['VersionInformationSize'] = len(version_infos.keys())
    except AttributeError:
        res['VersionInformationSize'] = 0

    return res

if __name__ == '__main__':
    # Load the ANN model
    model = load_model('ANN_model.h5')

    # Extract information from the given file
    data = extract_infos(sys.argv[1])

    # Prepare the input features for prediction
    pe_features = [value for value in data.values()]
    pe_features = np.array(pe_features).reshape(1, -1)

    # Make predictions using the ANN model
    res = model.predict(pe_features)

    # Print the prediction result
    if res > 0.5:
        print(f"The file {os.path.basename(sys.argv[1])} is malicious")
    else:
        print(f"The file {os.path.basename(sys.argv[1])} is legitimate")


Writing Final_Testing.py


In [ ]:
!git clone https://github.com/iosifache/DikeDataset.git

Cloning into 'DikeDataset'...
remote: Enumerating objects: 11989, done.
remote: Counting objects: 100% (11989/11989), done.
remote: Compressing objects: 100% (9243/9243), done.
remote: Total 11989 (delta 2749), reused 11981 (delta 2741), pack-reused 0
Receiving objects: 100% (11989/11989), 2.38 GiB | 33.83 MiB/s, done.
Resolving deltas: 100% (2749/2749), done.
Updating files: 100% (11936/11936), done.


## Legitimate File Prediction

In [ ]:
%run Final_Testing.py /content/drive/MyDrive/SEM6_Project/Malicious_File/PE_Malicious_File_Detection/Test_Sample/sublime_text_build_4169_x64_setup.exe

1/1 [==============================] - 0s 98ms/step
The file sublime_text_build_4169_x64_setup.exe is legitimate


In [ ]:
%run Final_Testing.py /content/DikeDataset/files/benign/08805d42f83cab69e6b93d9717da43d4d61892942b2573e45ec0e97adc6b506e.exe

1/1 [==============================] - 0s 61ms/step
The file 08805d42f83cab69e6b93d9717da43d4d61892942b2573e45ec0e97adc6b506e.exe is legitimate


## Malicious File Prediction

In [ ]:
%run Final_Testing.py /content/DikeDataset/files/malware/0008d6d1dcbe359dab5780be87005be659ea93eae5878a5b6e6ba4ed2d889b09.exe

1/1 [==============================] - 0s 59ms/step
The file 0008d6d1dcbe359dab5780be87005be659ea93eae5878a5b6e6ba4ed2d889b09.exe is malicious


In [ ]:
%run Final_Testing.py /content/DikeDataset/files/malware/0078e3961d934fcf2782d77812d2ba7177e23207099333a0cd084682ca0ef3ff.exe

1/1 [==============================] - 0s 61ms/step
The file 0078e3961d934fcf2782d77812d2ba7177e23207099333a0cd084682ca0ef3ff.exe is malicious


# FNN Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Split features and target variable
X = dataset.drop(['Name', 'md5', 'legitimate'], axis=1)
y = dataset['legitimate']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate the model on the test dataset
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
y_pred = np.squeeze(y_pred)  # Convert predictions to 1D array

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print evaluation metrics
print("Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Confusion Matrix:")
print(conf_matrix)


Epoch 1/20
1381/1381 [==============================] - 6s 3ms/step - loss: 0.0879 - accuracy: 0.9772 - val_loss: 0.0507 - val_accuracy: 0.9852
Epoch 2/20
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0588 - accuracy: 0.9858 - val_loss: 0.0460 - val_accuracy: 0.9865
Epoch 3/20
1381/1381 [==============================] - 4s 3ms/step - loss: 0.0512 - accuracy: 0.9868 - val_loss: 0.0442 - val_accuracy: 0.9870
Epoch 4/20
1381/1381 [==============================] - 4s 3ms/step - loss: 0.0515 - accuracy: 0.9876 - val_loss: 0.0413 - val_accuracy: 0.9876
Epoch 5/20
1381/1381 [==============================] - 4s 3ms/step - loss: 0.0442 - accuracy: 0.9877 - val_loss: 0.0397 - val_accuracy: 0.9880
Epoch 6/20
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0404 - accuracy: 0.9885 - val_loss: 0.0389 - val_accuracy: 0.9885
Epoch 7/20
1381/1381 [==============================] - 3s 2ms/step - loss: 0.0383 - accuracy: 0.9886 - val_loss: 0.0385 - val_accuracy:

# CNN Model

In [ ]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout

# Assuming your dataset is stored in a DataFrame called 'dataset'
# Assuming your dataset is stored in a DataFrame called 'dataset'

# Reshape data for CNN
X_cnn_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_cnn_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the CNN model architecture
model_cnn = Sequential([
    Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history_cnn = model_cnn.fit(X_cnn_train, y_train, epochs=20, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate the model on the test dataset
y_pred_prob_cnn = model_cnn.predict(X_cnn_test)
y_pred_cnn = (y_pred_prob_cnn > 0.5).astype(int)
y_pred_cnn = np.squeeze(y_pred_cnn)  # Convert predictions to 1D array

# Calculate evaluation metrics
accuracy_cnn = accuracy_score(y_test, y_pred_cnn)
precision_cnn = precision_score(y_test, y_pred_cnn)
recall_cnn = recall_score(y_test, y_pred_cnn)
f1_cnn = f1_score(y_test, y_pred_cnn)
conf_matrix_cnn = confusion_matrix(y_test, y_pred_cnn)

# Print evaluation metrics
print("Evaluation Metrics for CNN:")
print("Accuracy:", accuracy_cnn)
print("Precision:", precision_cnn)
print("Recall:", recall_cnn)
print("F1 Score:", f1_cnn)
print("Confusion Matrix:")
print(conf_matrix_cnn)

Epoch 1/20
1381/1381 [==============================] - 16s 11ms/step - loss: 0.0637 - accuracy: 0.9810 - val_loss: 0.0512 - val_accuracy: 0.9858
Epoch 2/20
1381/1381 [==============================] - 15s 11ms/step - loss: 0.0422 - accuracy: 0.9867 - val_loss: 0.0452 - val_accuracy: 0.9868
Epoch 3/20
1381/1381 [==============================] - 15s 11ms/step - loss: 0.0403 - accuracy: 0.9878 - val_loss: 0.0401 - val_accuracy: 0.9873
Epoch 4/20
1381/1381 [==============================] - 15s 11ms/step - loss: 0.0362 - accuracy: 0.9890 - val_loss: 0.0373 - val_accuracy: 0.9890
Epoch 5/20
1381/1381 [==============================] - 15s 11ms/step - loss: 0.0344 - accuracy: 0.9890 - val_loss: 0.0364 - val_accuracy: 0.9886
Epoch 6/20
1381/1381 [==============================] - 16s 11ms/step - loss: 0.0333 - accuracy: 0.9894 - val_loss: 0.0348 - val_accuracy: 0.9887
Epoch 7/20
1381/1381 [==============================] - 14s 10ms/step - loss: 0.0321 - accuracy: 0.9896 - val_loss: 0.0348 -